In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from datetime import datetime
from config import weather_api_key

In [2]:
# create a set of random lat and long combinations
lats = np.random.uniform(low = -90, high = 90, size = 1500)
lngs = np.random.uniform(low = -180, high = 180, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# city list
cities = []

# gets nearest city for each lat/long combo
for coords in coordinates:
    city = citipy.nearest_city(coords[0], coords[1]).city_name
    
    # if the city is unique, then add to list
    if city not in cities:
        cities.append(city)

# print count to confirm sufficient
len(cities)

601

In [5]:
# basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# list of city data
city_data = []

# print beginning of the logging
print('Beginning Data Retrieval     ')
print('-----------------------------')

# counters
record_count = 1
set_count = 1

# loop through all cities in list
for i, city in enumerate(cities):
    
    # group cities in sets of 50 for logging purposes
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    
    # endpoint URL for ech city
    city_url = url + '&q=' + city.replace(' ', '+')
    
    # log the url, record, and set numbers for the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    
    # add 1 to record count
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | bethel
Processing Record 2 of Set 1 | torbay
Processing Record 3 of Set 1 | lillooet
Processing Record 4 of Set 1 | new norfolk
Processing Record 5 of Set 1 | duluth
Processing Record 6 of Set 1 | vaini
Processing Record 7 of Set 1 | saint-philippe
Processing Record 8 of Set 1 | namatanai
Processing Record 9 of Set 1 | wahran
City not found. Skipping...
Processing Record 10 of Set 1 | port elizabeth
Processing Record 11 of Set 1 | labuhan
Processing Record 12 of Set 1 | ancud
Processing Record 13 of Set 1 | la palma
Processing Record 14 of Set 1 | cabo san lucas
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | khatanga
Processing Record 17 of Set 1 | castro
Processing Record 18 of Set 1 | ushuaia
Processing Record 19 of Set 1 | geraldton
Processing Record 20 of Set 1 | vestmanna
Processing Record 21 of Set 1 | georgetown
Processing Record 22 of Set 1 | yellowknife
Processin

Processing Record 39 of Set 4 | faanui
Processing Record 40 of Set 4 | krasnoyarsk-45
City not found. Skipping...
Processing Record 41 of Set 4 | nouakchott
Processing Record 42 of Set 4 | sabha
Processing Record 43 of Set 4 | puerto del rosario
Processing Record 44 of Set 4 | yarada
Processing Record 45 of Set 4 | paita
Processing Record 46 of Set 4 | belushya guba
City not found. Skipping...
Processing Record 47 of Set 4 | elk city
Processing Record 48 of Set 4 | brownsville
Processing Record 49 of Set 4 | esna
Processing Record 50 of Set 4 | birjand
Processing Record 1 of Set 5 | avera
Processing Record 2 of Set 5 | hobart
Processing Record 3 of Set 5 | kawalu
Processing Record 4 of Set 5 | vaitupu
City not found. Skipping...
Processing Record 5 of Set 5 | ketchikan
Processing Record 6 of Set 5 | onega
Processing Record 7 of Set 5 | najran
Processing Record 8 of Set 5 | aconibe
Processing Record 9 of Set 5 | shieli
Processing Record 10 of Set 5 | saskylakh
Processing Record 11 of Se

Processing Record 28 of Set 8 | nyurba
Processing Record 29 of Set 8 | yanan
City not found. Skipping...
Processing Record 30 of Set 8 | veraval
Processing Record 31 of Set 8 | dunedin
Processing Record 32 of Set 8 | beloha
Processing Record 33 of Set 8 | samusu
City not found. Skipping...
Processing Record 34 of Set 8 | dicabisagan
Processing Record 35 of Set 8 | vanavara
Processing Record 36 of Set 8 | ust-bolsheretsk
City not found. Skipping...
Processing Record 37 of Set 8 | adrar
Processing Record 38 of Set 8 | pangkalanbuun
Processing Record 39 of Set 8 | fairbanks
Processing Record 40 of Set 8 | bejar
Processing Record 41 of Set 8 | mitsamiouli
Processing Record 42 of Set 8 | petropavlovsk-kamchatskiy
Processing Record 43 of Set 8 | balakhninskiy
Processing Record 44 of Set 8 | plavinas
Processing Record 45 of Set 8 | esperance
Processing Record 46 of Set 8 | provideniya
Processing Record 47 of Set 8 | nyimba
Processing Record 48 of Set 8 | tawang
Processing Record 49 of Set 8 |

Processing Record 21 of Set 12 | ust-kamchatsk
City not found. Skipping...
Processing Record 22 of Set 12 | sur
Processing Record 23 of Set 12 | buala
Processing Record 24 of Set 12 | bichura
Processing Record 25 of Set 12 | wainwright
Processing Record 26 of Set 12 | cayenne
Processing Record 27 of Set 12 | guerrero negro
Processing Record 28 of Set 12 | bargal
City not found. Skipping...
Processing Record 29 of Set 12 | jaisalmer
Processing Record 30 of Set 12 | steamboat springs
Processing Record 31 of Set 12 | waiuku
Processing Record 32 of Set 12 | hargeysa
Processing Record 33 of Set 12 | yulara
Processing Record 34 of Set 12 | meulaboh
Processing Record 35 of Set 12 | pacific grove
Processing Record 36 of Set 12 | mon
Processing Record 37 of Set 12 | qasigiannguit
Processing Record 38 of Set 12 | qitaihe
Processing Record 39 of Set 12 | alice springs
Processing Record 40 of Set 12 | kijang
Processing Record 41 of Set 12 | kargil
Processing Record 42 of Set 12 | yining
Processing

In [10]:
len(city_data)

561